# Podcast_Reviews project
---

### We're eyeing an opportunity to diversify our advertising by sponsoring top podcasts. But here's the thing - we're not quite sure which ones would best align with our brand. We've got this “podcast_reviews” dataset from iTunes, and we're hoping it could shed some light

## 1. Profiling Categories

- We begin by checking for missing data in the categories table:
- The results indicate no missing podcast_id or category values, which is a positive start.

In [ ]:
SELECT
  COUNT(*) AS cnt_rows,
  SUM(CASE WHEN podcast_id IS NULL THEN 1 ELSE 0 END) AS cnt_missing_ids,
  SUM(CASE WHEN category IS NULL THEN 1 ELSE 0 END) AS cnt_missing_categories
FROM
  `podcast_reviews.categories`

- Next, we explore the diversity of the data in the categories table:

SELECT
 COUNT(*) AS cnt_rows,
 COUNT(DISTINCT podcast_id) AS cnt_distinct_podcasts,
 COUNT(DISTINCT category) AS cnt_distinct_categories
FROM
 `podcast_reviews.categories`

- The findings reveal more rows than unique podcasts, suggesting that podcasts may belong to multiple categories. To confirm this, we investigate further by counting the number of categories each podcast is associated with and by listing the top 5 in descending order.

SELECT podcast_id,
 COUNT(DISTINCT category) AS cnt_categories
FROM `podcast_reviews.categories`
GROUP BY podcast_id
ORDER BY cnt_categories DESC
LIMIT 5;

- This confirms that some podcasts are indeed associated with multiple categories. To understand the implications of this categorization, we select a few podcasts for a closer examination, revealing that a podcast can be linked to both broad and specific categories.

SELECT podcast_id, category
FROM `podcast_reviews.categories`
WHERE podcast_id IN ('c4be70db3c16d3607bd7ae4999cb6e86',
   'be905117ff9b3a13b1a065801241903b',
   'fa8c359031380cd4f3b38358c8a75f92')
ORDER BY podcast_id, category;

- We raise the hypothesis that every podcast_id that is associated with a lower level category such as “business-entrepreneurship” is also associated with the broader category such as “business”. To verify or to reject this hypothesis we take a closer look.

SELECT COUNT(DISTINCT podcast_id) AS cnt_distinct_podcasts,
 SUM(CASE WHEN category='business' THEN 1 ELSE 0 END) AS cnt_business
FROM `podcast_reviews.categories`
WHERE podcast_id IN (SELECT podcast_id FROM `podcast_reviews.categories` WHERE category='business-entrepreneurship')

- The results suggest that the majority of “business-entrepreneurship” associated podcasts were also associated with “business” (3371/3601 = 0.936 ~ 94 %), however this is not always the case.

Furthermore we observed that the podcasts are also associated with different categories such as “arts” and “business” at the same time.

This will make the evaluation much more difficult and error prone.

## 2. Profiling Podcasts

- Turning our attention to the podcasts table, our primary interest lies in ensuring that the podcast titles are present and there are no duplicate entries:
- The results confirm that each podcast has a unique ID and there are no missing values.

In [ ]:
SELECT COUNT(*) AS cnt_rows,
    COUNT(DISTINCT podcast_id) AS cnt_distinct_podcasts,
    SUM(CASE WHEN podcast_id IS NULL THEN 1 ELSE 0 END) AS cnt_nulls
FROM
 `podcast_reviews.podcasts`;

## 3. Profiling Reviews

- A brief examination of the reviews table aims to understand the volume of reviews per podcast and check for any missing podcast_id values:
- This analysis shows a healthy average number of reviews per podcast, with no missing podcast IDs, indicating active listener engagement across the board.

In [ ]:
SELECT COUNT(*) AS cnt_rows,
    COUNT(DISTINCT podcast_id) AS cnt_distinct_podcasts,
    ROUND(COUNT(*) / COUNT(DISTINCT podcast_id),1) AS ratio_review_per_podcast,
    SUM(CASE WHEN podcast_id IS NULL THEN 1 ELSE 0 END) AS cnt_nulls
FROM `podcast_reviews.reviews`;

## 4. Interim findings

- We explored the podcast_reviews database that provides data on podcasts in the USA and is based on scraped iTunes data. The database provides 16 irregular scrapings from period 2021-05 to 2023-02.

In [ ]:
SELECT
 DATE(MIN(run_at)) first_data_added,
 DATE(MAX(run_at)) last_data_added,
 COUNT(DISTINCT run_at) AS cnt_uploads
FROM `podcast_reviews.runs`;

# Review

Data reveals the multifaceted nature of podcast categorization within the dataset, with some podcasts appearing in multiple categories (such as “arts” and “business”), including higher level and lower level category ones (“business” and “business-entrepreneurship”) at the same time.

This complexity needs to be considered when analyzing the data at the category level as this introduces double-counting (multiple-counting) of reviews in overlapping categories. Thus we suggest abstaining from using this approach.

Thus we provide two tables:

1. Identifying the top 5 categories based on review volume and, within those, the top podcasts by reviews. This method will include double-counting and will not be precise. However, given the data structure, there is not much we can do about it. And its closest to the initial request.

2. Focusing on the most reviewed podcasts irrespective of their categories, identifying the top 25 podcasts based solely on review counts - could be also a valuable information for the decision makers. This approach offers a straightforward view of the most popular podcasts without the complications of categorization and double counting of reviews.

# Target

## Top 5 categories and top 5 podcasts

This query identifies the top 5 categories with the most reviews from the podcast_reviews.categories table, and within those categories, it finds the top 5 podcasts based on the number of reviews.

It uses two Common Table Expressions (CTEs): the first one (cat_reviews_count) to select the top categories, and the second one (top_podcasts_within_cat) to rank podcasts within these categories. The final SELECT statement then retrieves the category, the number of reviews for the category, podcast titles, and review counts for these top podcasts, ordering the results by category review count and individual podcast review count.

In [ ]:
WITH cat_reviews_count AS (
 SELECT
   categories.category,
   COUNT(*) AS cnt_category_reviews
 FROM
   `podcast_reviews.categories` AS categories
 JOIN
   `podcast_reviews.reviews` AS reviews ON categories.podcast_id = reviews.podcast_id
 GROUP BY
   categories.category
 ORDER BY
   cnt_category_reviews DESC
 LIMIT 5
),

top_podcasts_within_cat AS (
 SELECT
   cat_reviews_count.category,
   cat_reviews_count.cnt_category_reviews,
   categories.podcast_id,
   COUNT(*) AS cnt_reviews,
   RANK() OVER (PARTITION BY cat_reviews_count.category ORDER BY COUNT(*) DESC) AS rank
 FROM
   `podcast_reviews.categories` AS categories
 JOIN
   `podcast_reviews.reviews` AS reviews ON categories.podcast_id = reviews.podcast_id
 INNER JOIN
   cat_reviews_count AS cat_reviews_count ON categories.category = cat_reviews_count.category
 GROUP BY
   cat_reviews_count.category, cat_reviews_count.cnt_category_reviews, categories.podcast_id
)

SELECT
 top_podcasts_within_cat.category,
 top_podcasts_within_cat.cnt_category_reviews,
 podcasts.title,
 top_podcasts_within_cat.cnt_reviews AS review_count
FROM
 top_podcasts_within_cat
LEFT JOIN
 `podcast_reviews.podcasts` AS podcasts ON top_podcasts_within_cat.podcast_id = podcasts.podcast_id
WHERE
 top_podcasts_within_cat.rank <= 5
ORDER BY
 top_podcasts_within_cat.cnt_category_reviews DESC, review_count DESC;

## Summary

The table summarizes the top 5 podcasts within the top 5 categories based on the total number of reviews. The categories featured are "Society & Culture," "Comedy," "Education," "Business," and "News." Each category has an aggregate review count, with individual podcasts listed alongside their specific review counts. For instance, the "Society & Culture" category leads with 441,874 reviews across its top 5 podcasts, with "True Crime Obsessed" being the most reviewed podcast in this category with 7,310 reviews. Similarly, other categories follow this pattern, showcasing the diversity in content and audience engagement across different genres.

# Top 25 podcasts across all categories

This query compiles a list of the top 25 podcasts based on the number of reviews in the podcast_reviews.reviews table.

It creates a Common Table Expression (CTE) named top_25_podcasts that aggregates review counts for each podcast and selects the 25 with the highest counts. The final SELECT statement then joins this CTE with the podcast_reviews.podcasts table to retrieve the titles of these top podcasts along with their review counts.

In [ ]:
WITH top_25_podcasts AS (SELECT podcast_id, COUNT(*) AS cnt_reviews FROM `podcast_reviews.reviews`
  GROUP BY podcast_id
  ORDER BY cnt_reviews DESC
  LIMIT 25)

SELECT podcasts.title, top_25_podcasts.cnt_reviews
FROM top_25_podcasts
LEFT JOIN `podcast_reviews.podcasts` AS podcasts ON top_25_podcasts.podcast_id = podcasts.podcast_id;

The table provides a summary of the top 25 podcasts ranked by their review counts. "Crime Junkie" leads the list with 33,104 reviews, indicating its popularity. The genres vary widely, including true crime, comedy, education, and news, reflecting a diverse range of listener interests. Notably, "My Favorite Murder" and "Wow in the World" follow "Crime Junkie" with significant review counts, highlighting their strong listener engagement. The list also features well-known podcasts like "The Ben Shapiro Show," "The Joe Rogan Experience," and "Stuff You Should Know," showcasing their broad appeal. This summary underscores the wide appeal of these podcasts across different audience segments.